In [2]:
from __future__ import print_function, division
import time, os
import numpy as np
import matplotlib.pyplot as plt
import sys
import networkx as nx
import pandas

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [5]:
# 6/14/2016 20:17:07
s_to_date = lambda x: pandas.to_datetime(x, format='%m/%d/%Y %H:%M:%S')
snapshots_df = pandas.read_csv('/home/cs231n/data/Field_Snaps_With_warranty.txt', converters={'Event DateTime':s_to_date})

In [8]:
# 2/3/2016
r_to_date = lambda x: pandas.to_datetime(x, format='%m/%d/%Y')
repairs_df = pandas.read_csv('/home/cs231n/data/repairs.csv', converters={'Rpr_Dt':r_to_date})

In [25]:
selected_repairs = repairs_df[[
 'Chassis\nReference\nNumber',
 'Model Vehicle',
 'Build_Dt',
 'Dlvry_Dt',
 'In Service Date',
 'Miles',
 'Rpr_Dt',
 'ATA3',
 'ATA3Desc',
 'ATA6',
 'ATA6Desc',
 'ATA9',
 'ATA9Desc',
 'Fail Type',
 'Repair Cost']]
selected_repairs = selected_repairs[selected_repairs['Chassis\nReference\nNumber'].notnull()]

In [27]:
high_repairs = selected_repairs[selected_repairs['Repair Cost'].isin(['medium', 'high', 'very high'])]
veh_ids = high_repairs['Chassis\nReference\nNumber'].unique()

In [32]:
selected_repairs = high_repairs['Chassis\nReference\nNumber'].isin(veh_ids)

In [28]:
selected_snapshots = snapshots_df['Veh Ref ID'].isin(veh_ids)
selected_snapshots = snapshots_df[['Veh Ref ID',
 'Event DateTime',
 'Event Type Description',
 'Acc Pedal Position',
 'Ambient Air Temp',
 'Barometric Press',
 'Brake Switch',
 'Bus Utilization',
 'Cat Intake Gas Temp',
 'Cat Outlet Gas Temp',
 'Clutch Switch',
 'Cmd Eng Fuel Press',
 'Cruise Status',
 'Dpf Regen Inhibit Sw',
 'Dpf Thermal Mngmnt',
 'Drvr Demand Torque',
 'Eng Air Flow Rate',
 'Eng Avg Fuel Econ',
 'Eng Coolant Level',
 'Eng Coolant Temp',
 'Eng Demand Torque',
 'Eng DPF Intake Press',
 'Eng Egr Valve Pos',
 'Eng Exhaust Gas Temp',
 'Eng Fuel Del Press',
 'EngFuelTemp1',
 'Engine Speed',
 'Eng Man Abs Pressure',
 'Eng Oil Pressure',
 'EngInjRail1Press',
 'EngIntakeMan1Temp',
 'EngOilTemp1',
 'Eng Percent Torque',
 'EngTurbo1Boost',
 'EngTurbo1Pos',
 'EngTurbo1Speed',
 'Event - All Lamps On Time Hr',
 'Event - Amber Lamp Time Hr',
 'Event - Mil Lamp Time Hr',
 'Event - Red Lamp Time Hr',
 'Exhaust Tank Level',
 'Exhaust Tank Temp',
 'Fan Speed',
 'Keyswitch Bat Pot',
 'Part Trap Diff Press',
 'Part Trap Out Temp',
 'Scr Intake Gas Temp',
 'Scr Outlet Gas Temp',
 'Vehicle Speed',
 'Population',
 'DTCID',
 'Trip Distance',
 'Trip Idle Time',
 'Trip Run Time',
 'Altitude',
 'Engine Start Ambient',
 'Engine Start Coolant',
 'Ignition Cycle Counter',
 'Latitude',
 'Longitude',
 'Lifetime Idle Hours',
 'Lifetime Idle Fuel',
 'Lifetime Fuel',
 'Lifetime Distance',
 'Lifetime Engine Hours']]

In [35]:
def get_012_dates(end_date):
    ## split into 0 (10+), 1 (5-10), 2(0-5)
    two_end = end_date
    one_end = pandas.to_datetime(two_end) + pandas.DateOffset(days=-5)
    zero_end = one_end + pandas.DateOffset(days=-5)
    return (zero_end, one_end, two_end)

In [36]:
def get_repair_slices(veh_ids, snapshots, repairs, code='ATA9'):
    repair_slices = {}
    for veh_id in veh_ids:
        v_snapshots = snapshots[snapshots['Veh Ref ID'] == veh_id].sort_values(by='Event DateTime')
        v_repairs = repairs[repairs['Chassis\nReference\nNumber'] == veh_id].sort_values(by='Rpr_Dt')

        start_date = pandas.to_datetime('1/1/2000') ## in past so first snapshot is captured
        event_dt_key = 'Event DateTime'

        repair_slices[veh_id] = {}
        veh_slices = repair_slices[veh_id]

        ## Best indicator of repair type is the ATA9 code
        ## Iterate over each repair type and append slices
        for repair_type, repair_group in veh_repairs.groupby([code]):
            start = start_date
            end = -1

            ## dates of all of the repairs in that group
            repair_dates = repair_group.sort_values(by='Rpr_Dt')['Rpr_Dt']
            r_size = len(repair_dates)

            ## for each repair type, grab slices of snapshots
            veh_slices_repair = {0:[],1:[],2:[]}
            for repair in repair_group.itertuples():
                end = repair[7] ##['Rpr_Dt']

                (zero_end, one_end, two_end) = get_012_dates(end)
                
                # divide up snapshots into 0, 1, 2 slices
                two_mask = (v_snapshots[event_dt_key] > one_end) & (v_snapshots[event_dt_key] <= end)
                one_mask = (v_snapshots[event_dt_key] > zero_end) & (v_snapshots[event_dt_key] <= one_end)
                zero_mask = (v_snapshots[event_dt_key] >= start) & (v_snapshots[event_dt_key] <= zero_end)
                two_slices = v_snapshots.loc[two_mask]
                one_slices = v_snapshots.loc[one_mask]
                zero_slices = v_snapshots.loc[zero_mask]

                if len(two_slices) > 0:
                    veh_slices_repair[2].append(two_slices)
                if len(one_slices) > 0:
                    veh_slices_repair[1].append(one_slices)
                if len(zero_slices) > 0:
                    veh_slices_repair[0].append(zero_slices)

                ## reset start to end for next iteration
                start = end
            
            if len(veh_slices_repair[0]) > 0 or len(veh_slices_repair[1]) > 0 or len(veh_slices_repair[2]) > 0:
                veh_slices[repair_type] = veh_slices_repair

    return repair_slices

In [ ]:
repair_slices_all = get_repair_slices(veh_ids, selected_snapshots, selected_repairs)